https://kerneld.tistory.com/32
위 문서를 참고해서 ColBERT 콘다 환경을 셋팅하자.

In [1]:
#!pip install openai

In [1]:
# 학습된 모델을 허깅페이스에 저장/로드 할지 여부.
is_use_hf_store_load_trained_model = True

if is_use_hf_store_load_trained_model:
    hf_repo_id = 'sentence-transformer-klue-temp'
    hf_full_repo_id = f"Kerneld/{hf_repo_id}"

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [3]:
import json

with open("../../data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]

# questions_from_contents.jsonl 파일은 Generate_questions_from_content.ipynb 를 실행하면 생성됨.
with open("questions_from_contents.jsonl") as f:
    qfcs = [json.loads(line) for line in f]

In [4]:
# docs 에서 줄바꿈 문자를 제거 (학습 할때 에러 방지)
for doc in docs:
    doc['content'] = doc['content'].replace("\n", "")
    doc['content'] = doc['content'].replace("\r", "")

In [ ]:
print(docs[0])
print(qfcs[0])

In [ ]:
docs_only_content = [doc['content'] for doc in docs]
docs_only_content[0]

In [ ]:
qfcs_only_question = [qfc['question'] for qfc in qfcs]
qfcs_only_question[0]

In [8]:
# 필요한 라이브러리 import

import colbert
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

## 1. Indexing

빠른 실습을 위해 처음 2000개의 구절만 색인

In [9]:
# 기본적인 상수 정의config

nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300 # truncate passages at 300 tokens
experiment = 'ir_contest'

한국어 데이터에 대한 색인/검색

In [ ]:
#checkpoint = 'colbert-ir/colbertv2.0'
checkpoint = 'hunkim/sentence-transformer-klue'
#checkpoint = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
#checkpoint = "google-bert/bert-base-multilingual-cased"
index_name = 'science_common_sense'

with Run().context(RunConfig(nranks = 1, experiment = experiment)):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(doc_maxlen = doc_maxlen, nbits = nbits, kmeans_niters = 4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                # Consider larger numbers for small datasets.
    indexer = Indexer(checkpoint = checkpoint, config = config)
    indexer.index(name = index_name, collection = docs_only_content, overwrite = True)

In [10]:
def search_docs(query, experiment, index_name, collection, k):
    with Run().context(RunConfig(experiment = experiment)):
        searcher = Searcher(index = index_name, collection = collection)

    # Find the top-3 passages for this query
    results = searcher.search(query, k)
    
    ret = []

    # Print out the top-k retrieved passages
    for passage_id, passage_rank, passage_score in zip(*results):
        print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")
        
        dic = {
            'score': passage_score,
            'passage_id': passage_id,
            'content': searcher.collection[passage_id],
        }
        ret.append(dic)
    
    return ret

In [ ]:
ret = search_docs('나무의 분류에 대해 조사해 보기 위한 방법은?', experiment, index_name, docs_only_content, 3)
print(ret)

## 3. Training TODO

LLM을 활용한 생성데이터로 ColBERT를 학습한 후 더 나은 성능의 모델을 활용하는 방법 실습

In [ ]:
import random

triples_data = []
max_c_idx = len(docs_only_content) - 1

for q_idx in range(len(qfcs_only_question)):
    # 관련 있는 문서 idx
    c_idx = qfcs[q_idx]['docOffset']

    # 관련 없는 문서 idx 결정
    mc_idx = random.randint(0, max_c_idx)
    while mc_idx == c_idx:
        mc_idx = random.randint(0, max_c_idx)

    triples_data.append(f'{q_idx}, {c_idx}, {mc_idx}')
    print(f"({q_idx}, {c_idx}, {mc_idx}) question: {qfcs_only_question[q_idx]}, relevance doc: {docs_only_content[c_idx]}, not relevance doc: {docs_only_content[mc_idx]}")

print(triples_data)

In [13]:
# ColBERT 학습을 위하여 학습 데이터를 파일에 저장
collection_file = 'collection.tsv'
query_file = 'query.tsv'
triples_file = 'triples'

with open(collection_file, 'w') as f:
  for i,item in enumerate(docs_only_content):
    f.write(f'{i}\t{item}\n')

with open(query_file, 'w') as f:
  for i,item in enumerate(qfcs_only_question):
    f.write(f'{i}\t{item}\n')

with open(triples_file, 'w') as f:
  for i,item in enumerate(triples_data):
    f.write(f'[{item}]\n')

### 3.2 새로 만든 데이터로 모델 학습

In [ ]:
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert import Trainer

if __name__=='__main__':
    with Run().context(RunConfig(nranks=1, experiment="training")):

        config = ColBERTConfig(
            bsize=24,
            root="./experiments",
        )

        trainer = Trainer(
            triples=triples_file,
            queries=query_file,
            collection=collection_file,
            config=config,
        )

        # Pretrained model을 한국어 기반 모델로 설정해 준다.
        checkpoint_path = trainer.train(checkpoint='hunkim/sentence-transformer-klue')
        #checkpoint_path = trainer.train(checkpoint="snunlp/KR-SBERT-V40K-klueNLI-augSTS")
        #checkpoint_path = trainer.train(checkpoint="google-bert/bert-base-multilingual-cased")
        #checkpoint_path = trainer.train()

        print(f"Saved checkpoint to {checkpoint_path}...")

In [ ]:
# # 학습된 모델의 위치
!find experiments/training -name colbert

학습된 모델을 허깅페이스에 업로드

In [ ]:
from huggingface_hub import login
from huggingface_hub import HfApi

if is_use_hf_store_load_trained_model:
    model_save_path = 'experiments/training/none/2024-10/16/05.29.48/checkpoints/colbert'
    
    login(token=os.getenv('HF_TOKEN'))
    
    api = HfApi()
    api.create_repo(repo_id=hf_repo_id)

    api.upload_folder(
        folder_path=model_save_path,
        repo_id=hf_full_repo_id,
        repo_type="model",
    )

학습된 모델로 다시 색인 및 검색

In [ ]:
# 위에서 확인한 학습된 모델의 위치를 checkpoint에 넣어줌
if is_use_hf_store_load_trained_model:
    checkpoint = hf_full_repo_id
else:
    checkpoint = 'experiments/training/none/2024-10/16/12.01.30/checkpoints/colbert'
    
experiment = 'after_trained'
index_name = 'science_common_sense'

with Run().context(RunConfig(nranks=1, experiment=experiment)):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(nbits=nbits, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                # Consider larger numbers for small datasets.
    indexer = Indexer(checkpoint = checkpoint, config = config)
    indexer.index(name = index_name, collection = docs_only_content, overwrite=True)

In [ ]:
ret = search_docs('나무의 분류에 대해 조사해 보기 위한 방법은?', experiment, index_name, docs_only_content, 3)
print(ret)

학습된 모델로 추론을 해보자~

In [19]:
from openai import OpenAI
import traceback

# OpenAI API 키를 환경변수에 설정
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
#os.environ["OPENAI_API_KEY"] = "Your API Key"

client = OpenAI()
# 사용할 모델을 설정(여기서는 gpt-3.5-turbo-1106 모델 사용)
llm_model = "gpt-3.5-turbo-1106"

In [20]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 대화를 통해 과학 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
- search api를 호출할 때, 사용자의 대화가 멀티턴이 아니라면, 사용자의 대화 내용을 그대로 standalone_query 에 대입한다.
- 과학 상식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.
"""

# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "search relevant documents",
            "parameters": {
                "properties": {
                    "standalone_query": {
                        "type": "string",
                        "description": "Final query suitable for use in search from the user messages history."
                    }
                },
                "required": ["standalone_query"],
                "type": "object"
            }
        }
    },
]


In [21]:
# LLM과 검색엔진을 활용한 RAG 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            tools=tools,
            #tool_choice={"type": "function", "function": {"name": "search"}},
            temperature=0,
            seed=1,
            timeout=10
        )
    except Exception as e:
        traceback.print_exc()
        return response

    # 검색이 필요한 경우 검색 호출후 결과를 활용하여 답변 생성
    if result.choices[0].message.tool_calls:
        tool_call = result.choices[0].message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        standalone_query = function_args.get("standalone_query")
        response["standalone_query"] = standalone_query

        hits_merged_list = search_docs(standalone_query, experiment, index_name, docs_only_content, 3)
        
        retrieved_context = []
        for i, rst in enumerate(hits_merged_list):
            if i >= 3:
                break   # 최대 3개까지만..
            
            doc = docs[rst['passage_id']]
            
            retrieved_context.append(doc["content"])
            
            response["topk"].append(doc["docid"])
            response["references"].append({"score": rst["score"], "content": doc["content"]})

        content = json.dumps(retrieved_context)
        messages.append({"role": "assistant", "content": content})
        msg = [{"role": "system", "content": persona_qa}] + messages
        try:
            qaresult = client.chat.completions.create(
                    model=llm_model,
                    messages=msg,
                    temperature=0,
                    seed=1,
                    timeout=30
                )
        except Exception as e:
            traceback.print_exc()
            return response
        response["answer"] = qaresult.choices[0].message.content

    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content

    return response

In [22]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            # if idx > 0:
            #     break
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1


In [ ]:
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
submission_filename = "submission_016-4.csv"

eval_rag("../../data/eval.jsonl", submission_filename)